In [1]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [2]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [3]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [4]:
model_tag = 'llama3'  
dataset_tag = 'go_emotions'
prompt = 'random_samples'
temp = 0
number_of_experimet = 1
selected_samples =  'cot_unstable'

In [5]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{model_tag}/{dataset_tag}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}_42.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [6]:
dataset_for_annotation.columns

Index(['text', 'label'], dtype='object')

In [7]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

Llama3_1LLM
Params: {}                                                    text        label
0     Ok.... he can *call* it the State of the Union...  disapproval
1                                 Wheres the telescope?    curiosity
2     > Why create more jobs when no one is availabl...    confusion
3        [NAME] deserves to play for a proper club ffs.   admiration
4     I'm sorry, do people NOT listen to [NAME] 9th ...    gratitude
...                                                 ...          ...
1915  Yeah, i dont know bout you guys, but my parent...     approval
1916  This is hilarious! Sounds similar to my buddy'...    amusement
1917                   But that jumper looks hella cosy          joy
1918                  Sorry. This thing knows whats up!      remorse
1919  As a household we deal with it by respecting e...     approval

[1920 rows x 2 columns] Task: Read the following text and determine the most appropriate category based on its content and the examples below.

Cate

In [8]:
text_col = "text"
label_col = "label"
output_col = "output"

In [9]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [10]:
df = annotator.get_results(is_save=True, output_path=output_path)

/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)
Process Process-2:
Traceback (most recent call last):
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py", line 13, in call_chain_process
    result = chain.invoke({"text": text})
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/langchain/chains/base.py", line 170, in invoke
    raise e
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/site-pac

KeyboardInterrupt: 

  File "/home/julita/anaconda3/envs/anote/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/ssl.py", line 1251, in recv_into
    return self.read(nbytes, buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/ssl.py", line 1103, in read
    return self._sslobj.read(len, buffer)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/random_samples_temp0/gpt4omini/go_emotions_gpt4omini_random_samples_cot_unstable_temp0_exp1.csv


In [ ]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label,error
0,Ok.... he can *call* it the State of the Union...,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",disapproval,NaN
1,Wheres the telescope?,curiosity,"{'content': [{'token': 'cur', 'bytes': [99, 11...","[{'token': 'cur', 'bytes': [99, 117, 114], 'lo...",curiosity,NaN
2,> Why create more jobs when no one is availabl...,neutral,"{'content': [{'token': 'neutral', 'bytes': [11...","[{'token': 'neutral', 'bytes': [110, 101, 117,...",confusion,NaN
3,[NAME] deserves to play for a proper club ffs.,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",admiration,NaN
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",confusion,"{'content': [{'token': 'conf', 'bytes': [99, 1...","[{'token': 'conf', 'bytes': [99, 111, 110, 102...",gratitude,NaN
...,...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",approval,NaN
1916,This is hilarious! Sounds similar to my buddy'...,amusement,"{'content': [{'token': 'am', 'bytes': [97, 109...","[{'token': 'am', 'bytes': [97, 109], 'logprob'...",amusement,NaN
1917,But that jumper looks hella cosy,approval,"{'content': [{'token': 'approval', 'bytes': [9...","[{'token': 'approval', 'bytes': [97, 112, 112,...",joy,NaN
1918,Sorry. This thing knows whats up!,amusement,"{'content': [{'token': 'am', 'bytes': [97, 109...","[{'token': 'am', 'bytes': [97, 109], 'logprob'...",remorse,NaN


In [ ]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df.to_csv(path, index=False)
    
    return df


In [ ]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,error,was_in_selected_samples
0,Ok.... he can *call* it the State of the Union...,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",disapproval,NaN,False
1,Wheres the telescope?,curiosity,"{'content': [{'token': 'cur', 'bytes': [99, 11...","[{'token': 'cur', 'bytes': [99, 117, 114], 'lo...",curiosity,NaN,False
2,> Why create more jobs when no one is availabl...,neutral,"{'content': [{'token': 'neutral', 'bytes': [11...","[{'token': 'neutral', 'bytes': [110, 101, 117,...",confusion,NaN,False
3,[NAME] deserves to play for a proper club ffs.,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",admiration,NaN,False
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",confusion,"{'content': [{'token': 'conf', 'bytes': [99, 1...","[{'token': 'conf', 'bytes': [99, 111, 110, 102...",gratitude,NaN,False
...,...,...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",approval,NaN,False
1916,This is hilarious! Sounds similar to my buddy'...,amusement,"{'content': [{'token': 'am', 'bytes': [97, 109...","[{'token': 'am', 'bytes': [97, 109], 'logprob'...",amusement,NaN,False
1917,But that jumper looks hella cosy,approval,"{'content': [{'token': 'approval', 'bytes': [9...","[{'token': 'approval', 'bytes': [97, 112, 112,...",joy,NaN,False
1918,Sorry. This thing knows whats up!,amusement,"{'content': [{'token': 'am', 'bytes': [97, 109...","[{'token': 'am', 'bytes': [97, 109], 'logprob'...",remorse,NaN,False
